In [26]:
%run 1_Objective_function.ipynb

EX_asp__L_e EX_asp__L_e: asp__L_e -->  L-Aspartate exchange
ASPtex ASPtex: asp__L_e <=> asp__L_p L-aspartate transport via diffusion (extracellular to periplasm)
0.87699721425716
0.87392
4.596027557488181


In [3]:
#pip freeze

To utilize the Co-factor swap function available in cameo we must first instal specific versions of cobra, numpy and pandas. This is because certain functions in cameo have not been updated and therefore are only compatible with older versions of python. For this section a python envoronment running version 3.7.16 was utilized. 

In [6]:
!python -m pip  install cobra==0.29.0
!python -m pip  install cameo
!python -m pip numpy==1.26.1
!python -m pip  install pandas==2.1.2

  Using cached cobra-0.29.0-py2.py3-none-any.whl (1.2 MB)
  Using cached depinfo-2.2.0-py3-none-any.whl (24 kB)


ERROR: Ignored the following versions that require a different python version: 1.8.0 Requires-Python >=3.8; 1.8.1 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement optlang~=1.8 (from cobra) (from versions: 0.0.3, 0.0.4, 0.1.0, 0.1.2, 0.2.1, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.7, 0.2.8, 0.2.9, 0.2.13, 0.2.14, 0.2.15, 0.2.16, 0.2.17, 0.2.18, 0.2.19, 0.2.21, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.6.0b1, 0.6.0, 0.6.1, 0.6.2, 0.6.3, 0.6.4, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.2.4, 1.2.5, 1.3.0, 1.4.0a2, 1.4.0, 1.4.1, 1.4.2, 1.4.3, 1.4.4, 1.4.6, 1.4.7, 1.5.0, 1.5.1, 1.5.2, 1.6.0, 1.6.1, 1.7.0)
ERROR: No matching distribution found for optlang~=1.8


ERROR: unknown command "numpy==1.26.1"

ERROR: Ignored the following versions that require a different python version: 1.4.0 Requires-Python >=3.8; 1.4.0rc0 Requires-Python >=3.8; 1.4.1 Requires-Python >=3.8; 1.4.2 Requires-Python >=3.8; 1.4.3 Requires-Python >=3.8; 1.4.4 Requires-Python >=3.8; 1.5.0 Requires-Python >=3.8; 1.5.0rc0 Requires-Python >=3.8; 1.5.1 Requires-Python >=3.8; 1.5.2 Requires-Python >=3.8; 1.5.3 Requires-Python >=3.8; 2.0.0 Requires-Python >=3.8; 2.0.0rc0 Requires-Python >=3.8; 2.0.0rc1 Requires-Python >=3.8; 2.0.1 Requires-Python >=3.8; 2.0.2 Requires-Python >=3.8; 2.0.3 Requires-Python >=3.8; 2.1.0 Requires-Python >=3.9; 2.1.0rc0 Requires-Python >=3.9; 2.1.1 Requires-Python >=3.9; 2.1.2 Requires-Python >=3.9; 2.1.3 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement pandas==2.1.2 (from versions: 0.1, 0.2, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.5.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0, 0.10.1

In [4]:
#!python -m pip install ipykernel==6.26.0
#!python -m pip install ipyparallel==8.6.1
#!python -m pip install ipython==8.17.2
#!python -m pip install ipython-genutils==0.2.0

In [5]:
#!python -m pip  install --upgrade markupsafe
#!python -m pip  install --upgrade Jinja2

We can now procede to import all the necessary extensions. 

In [1]:
from cobra.io import read_sbml_model #importing functions needed
from cobra import Reaction, Metabolite
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

In [6]:
#pip install numpy==1.11

In [7]:
#!python -m pip install  markupsafe==2.0.1
#!python -m pip  install  nbformat
#!python -m pip  install sympy==1.12

First we want to determine a baseline to which compare all posible alterations done to the model. Once again we set the upper boung of the objective function for Aspartate to 80% of the ideal maximum value to better represent biological productivity levels and to allow for the algorithm to have room for improvement. 

In [63]:
with model:
    model.objective = model.reactions.BIOMASS_Ec_iML1515_core_75p37M
    base_max_growth = model.slim_optimize()
    print(base_max_growth)
    model.objective = model.reactions.EX_asp__L_e
    model.reactions.EX_asp__L_e.upper_bound=4.59*(0.8)
    base_max_production = model.slim_optimize()
    print(base_max_production)

0.87392
3.672


We generate variables for biomass and for aspartate in order for them to be considered as goals for the optimization process. The max_size value detemrines the scale of the analysis. 

In [59]:
with model:
 model.objective = model.reactions.EX_asp__L_e
 model.reactions.EX_asp__L_e.upper_bound=4.59*(0.8)
 biomass=  model.reactions.BIOMASS_Ec_iML1515_core_75p37M
 Aspartate_prod = product_yield(model.reactions.EX_asp__L_e, biomass) 
 swap_opt = CofactorSwapOptimization(model=model, objective_function = Aspartate_prod, plot=True)
 result = swap_opt.run(max_size=10)
     

Starting optimization at Thu, 30 Nov 2023 01:31:49


HBox()

Finished after 00:01:42


In [62]:
result

,index,targets,fitness


The algorith was unable to find any possible CO-factor swaps that would result in a higher aspartate producition. 